Standard UUIDs are nice and all, but why not extend the character range to include all standard ascii letters and numbers, also, get rid of embedded MAC addresses and time stamps.  

In [1]:
from secrets import token_urlsafe as TUS

class xuid:
    def __init__(self, TUS_length=64):
        """Initializes a string using secrets.token_urlsafe, this string is
            split into sections to imitate a uuid.

            Upon initialization it creates some of the same standard outputs
                as uuid, including urn (urn:xuid:{xuid}) and fields (tuple of
                values). 

        Args:
            TUS_length (int, optional): An int that is used in the 
                secrets.token_urlsafe (TUS) function to generate the random 
                string of characters. Defaults to 64. 
        """
        if TUS_length <= 0:
            raise ValueError("TUS_length cannot be less than or equal to 0. "
                             "An integer greater than or equal to 30 is "
                             "recommended, default is 64.")
        initial_string = TUS(TUS_length).replace("_", "").replace("-", "")
        while len(initial_string) <= 32:
            initial_string += TUS(TUS_length).replace("_", "").replace("-", "")
        self.used_chars = initial_string[:32]
        segments = (initial_string[0:8],
                    initial_string[8:12],
                    initial_string[12:16],
                    initial_string[16:20],
                    initial_string[20:32])
        self.xuid_join = "-".join(segments)
        self.fields = segments
        self.urn = f"urn:xuid:{self.xuid_join}"
    def __str__(self):
        """Returns a simple 36 character string in the same style as UUID
        """
        return(self.xuid_join)
    def __int__(self):
        """Returns an integer generated from the UTF-8 ordinal values of the 
            initial random string. 
        """
        return(self.shoint())
    def __index__(self):
        """Enables the return of hex and byte values using an integer generated 
            from the UTF-8 ordinal values of the initial random string.  
        """
        return(self.shoint())
    def __repr__(self):
        return(f"XUID('{self.xuid_join}')")
    def long_interizer(self):
        """A step by step approach to creating an integer from the randomly 
            generated string of letters from secrets.token_urlsafe.
            Steps:
            1. Convert string to list of chars
            2. Create list of ordinal values from list of chars
            3. Create list of binary from ordinal values
            4. Create list of strings from binary values
            5. Correct list of binary strings to meet 8-bits per binary string.
            6. Concatenate binary strings into longer binary string suitable
                for conversion using int() built in.
            7. Return int(long binary string)
        """
        charlist = list(self.used_chars)
        ordlist = [ord(x) for x in charlist]
        binlist = [bin(x) for x in ordlist]
        strbin = [str(x)[2:] for x in binlist]
        strbin8 = [f"{'0' * (8-len(x))}{x}" for x in strbin]
        bstr = "0b" + "".join(strbin8)
        return(int(bstr, 2))
    def short_interizer(self):
        """A short version of long_interizer() using list comprehension better.

            See the documentation for long_interizer for the steps to create 
                the integer.  

            Testing shows this is a faster method than long_interizer()
        """
        strlist = [str(bin(ord(x)))[2:] for x in list(self.used_chars)]
        return(int("0b" +  "".join(
                [f"{'0' * (8-len(x))}{x}" for x in strlist]), 2))
    def shoint(self):
        """An even shorter version of long_interizer() using list comprehension.
        
            See the documentation for long_interizer for the steps to create
                the integer.
                
            Testing shows this is a faster method than short_interizer()"""
        return(
            int("0b" +  "".join(
                [f"{'0'*8}{str(bin(ord(x)))[2:]}"[-8:] for x in list(self.used_chars)]
                               ), 2
            ))
        
    
print(str(xuid()))
print(xuid())
int(xuid())
hex(xuid())

x8SpJpyQ-3Cmd-9og1-Lea5-HjmYLChkR2IF
fUTdIytT-IgW2-BoPt-VxN1-MXtJH8gJrgLj


'0x434175754f4e356b326c74565a6e666d6e496d333043677450324b596a676c6a'

The object is very simple, just a random string from `token_urlsafe`, and then sliced up to a format we can use.
Now let's figure out which method of creating the __int__ is faster, with a lot of variables or our two list comprehensions.

In [2]:
import timeit

def sf():
    xuid().short_interizer()
def lf():
    xuid().long_interizer()
st = timeit.timeit(lambda: sf(), number=1000*100*100)
lt = timeit.timeit(lambda: lf(), number=1000*100*100)
print(f"Short Time:   {st}s\nLong Time:    {lt}s")
print(f"Short time is {(lt/st)*100 - 100:2.2f}% faster")

Short Time:   171.2818116s
Long Time:    190.40002690000003s
Short time is 11.16% faster


Okay, so we can see that the short interizer is the faster method. As this is just a random series of characters, there's no reason to reinstantiate an older xuid. 

***Update 2022-06-02***  
I did some more "optimization" and created an even faster method of generating the integer that could be a single line if I had ignored the PEP8 rule of 79 chars per line. 

In [3]:
def s():
    xuid().shoint()
sht = timeit.timeit(lambda: s(), number=1000*100*100)
print(f"Shoint Time: {sht}")
print(f"Shoint is {(lt/sht)*100 - 100:2.2f}% faster than long interizer")
print(f"Shoint is {(st/sht)*100 - 100:2.2f}% faster than short interizer")

Shoint Time: 162.60144479999997
Shoint is 17.10% faster than long interizer
Shoint is 5.34% faster than short interizer


Also laying the groundwork to test for XUID collisions by generating a ton of them, and then checking for duplicates. 

In [4]:
import string, os, glob

def dictinit():
    d = {}
    for char1 in list(string.ascii_letters + string.digits):
        for char2 in list(string.ascii_letters + string.digits):
            d[char1 + char2] = []
    return(d)

def test_write(testing_dir, d):
    for char in d.keys():
        with open(f"{testing_dir}{os.sep}{char}.txt", "a") as f:
            for x in d[char]:
                f.write(f"{x}\n".encode("ascii"))

def check_file(filepath):
    with open(filepath, "r") as f:
        L = f.split("\n")
    L = [x for x in L if len(x) == 36]
    S = list(set(L))
    if len(L) != len(S):
        print(f"{filepath} includes {len(L)-len(S)} collisions!")
    else:
        print("No collisions found.")
        
def collision_test(testing_dir, test_count=10_000_000_000, index_pause=10_000_000):
    for i in range(test_count):
        if i == 0:
            d = dictinit()
        if i % index_pause == 0 and i > 0:
            print(f"{i//1_000_000:^5}M - {i/test_count:>6}%")
            test_write(testing_dir, d)
            del d # hashtag memory management
            d = dictinit()
        x = str(xuid())
        d[x[:2]].append(x)
    for f in glob.glob(f"{testing_dir}{os.sep}*.txt"):
        check_file()
#testdir = r""
#collision_test(testdir)